In [1]:
from selenium import webdriver
from base.driver import SeleniumDriver
from lxml import html

driver = webdriver.Chrome()

# Kesehatan
https://health.kompas.com/?source=navbar


In [52]:
import time
class StorageArticle:
    def __init__(self) -> None:
        self.data_scraping = {}

    def add_category(self, category):
        if category not in self.data_scraping:
            self.data_scraping[category] = {"total":0, "date":"", "articles":[]}

    def add_article(self, category, index, title, link, content=[]):
        # Periksa apakah artikel dengan title dan link yang sama sudah ada
        if not any(article['title'] == title and article['link'] == link for article in self.data_scraping[category]["articles"]):
            article = {
                "index": index,
                "title": title,
                "link": link,
                "content": content,
                "published": False
            }
            self.data_scraping[category]["articles"].append(article)
            self.data_scraping[category]["total"] += 1
        else:
            print(f"Artikel '{title}' dengan link '{link}' sudah ada di kategori '{category}'.")


    def update_article(self, category, index, new_data):
        if category in self.data_scraping:
            for article in self.data_scraping[category]["articles"]:
                if article["index"] == index:
                    article.update(new_data)
                    print(f"Artikel pada index {index} berhasil diperbarui dengan data {new_data}.")
                    return
                print(f"Artikel dengan index {index} tidak ditemukan di kategori {category}.")
            else:
                print(f"Kategori '{category}' tidak ditemukan.")

    def get_data(self):
        return self.data_scraping




class Kompas(SeleniumDriver):
    def __init__(self, driver, data_storage):
        super().__init__(driver)
        self.driver = driver
        self.data_storage = data_storage

    _list_category = {
        "kesehatan":"https://health.kompas.com/?source=navbar",
        # "otomotif":"https://otomotif.kompas.com/?source=navbar",
        # "teknologi":"https://tekno.kompas.com/?source=navbar",
        # "gaya hidup":"https://lifestyle.kompas.com/?source=navbar",
        # "keuangan":"https://money.kompas.com/?source=navbar",
        # "masakan":"https://www.kompas.com/food?source=navbar",
        # "traveling":"https://travel.kompas.com/?source=navbar"
    }
    _xp_top_ten_news = "//div[@class='most__wrap clearfix'] / div"
    _xp_top_ten_link = ".//a/@href"
    _xp_top_ten_title = ".//h4/text()"
    _lxml_content_article = "//div[@class='read__content']"
    _target_tag = ["h1","h2","h3","h4","h5","h6","p","img"]

    def most_list_top_ten(self,category):
        list_el = self.getElementList(self._xp_top_ten_news,"xpath")
        for index, el in enumerate(list_el,start=1):
            # convert element ke lxml untuk hasil lebih dinammis
            el = self.convert_webElement_to_lxml(el)
            link = self.get_link(el)
            title = self.get_title(el)

            self.data_storage.add_article(category,index,title,link)

    def convert_webElement_to_lxml(self, webElement):
        return html.fromstring(webElement.get_attribute('innerHTML'))

    def get_link(self,element):
        link = element.xpath(self._xp_top_ten_link)[0]
        return link

    def get_title(self,element):
        title = element.xpath(self._xp_top_ten_title)[0]
        return title

    def get_driver(self):
        return driver

    def get_article(self, url):
        print(url)
        self.driver.get(url)
        time.sleep(10)

        xp = html.fromstring(driver.page_source)
        res_div = xp.xpath(self._lxml_content_article)[0]
        # print(html.tostring(res_div, pretty_print=True, encoding='unicode'))

        # span digunakaan untuk ads
        # p perlu di cek dalam nya jika ada attr target
        # div attr target skip
        exclude_attrib = ["kompasidRec"]
        content = []
        h1 = xp.xpath("//h1")[0]
        text = h1.text
        content.append({h1.tag:text})

        img= xp.xpath("//div[@class='photo__wrap'] / img")[0]
        src= img.xpath("./@src")[0]
        alt= img.xpath("./@alt")[0]

        content.append({"img":img.tag,"src":src,"alt":alt})
        for element in res_div.iter():
            tmp = {}
            tag = element.tag
            attr = element.attrib
            text = element.xpath("string(.)")

            if tag not in self._target_tag:
                continue
            if "Baca juga:" in text:
                continue

            tmp[tag] = text.replace("KOMPAS.com", "PT. XYZ")
            content.append(tmp)

        print(content)
        print("\n\n")
    def scraping_kompas(self):
        for category, link in self._list_category.items():
            self.driver.get(link)
            self.data_storage.add_category(category)
            self.most_list_top_ten(category)

s = StorageArticle()
k = Kompas(driver,s)
k.scraping_kompas()
driver = k.get_driver()
data = s.get_data()

for d in data:
    for article in data[d]['articles']:
        k.get_article(article['link'])

https://health.kompas.com/read/24K01060000968/adakah-tanda-peringatan-sebelum-stroke-ini-penjelasannya-
[{'h1': 'Adakah Tanda Peringatan Sebelum Stroke? Ini Penjelasannya...'}, {'img': <Element img at 0x7efd5c34e580>, 'src': 'https://asset.kompas.com/crops/VGp35FCm-bZzQUY6kAF5T2HnxUM=/0x0:500x333/1200x800/data/photo/2022/10/29/635d21f919a5a.jpg', 'alt': 'Ada kondisi yang bisa muncul sebagai tanda peringatan stroke, seperti sakit kepala dan mini stroke. Namun, tidak selalu stroke bisa diprediksi.'}, {'p': 'PT. XYZ - Stroke adalah penyebab kematian dan disabilitas tertinggi baik di dunia maupun di Indonesia.'}, {'p': 'Berdasarkan hasil Survei Kesehatan Indonesia (SKI) pada 2023, prevalensi stroke di Indonesia adalah 8,3 per 1.000 penduduk.'}, {'p': 'Sebelum stroke terjadi, bisa muncul kondisi sebagai tanda peringatan agar Anda waspada dan bisa melakukan pengobatan lebih dini.'}, {'p': 'Namun, serangan stroke tidak selalu dapat diprediksi.'}, {'p': 'Apa tanda peringatan sebelum stroke ter

In [17]:
driver.get("https://health.kompas.com/read/24J31180000568/apa-yang-terjadi-jika-makan-buah-anggur-setiap-hari-")

In [46]:
xp = html.fromstring(driver.page_source)
res_div = xp.xpath("//div[@class='read__content']")[0]
# print(html.tostring(res_div, pretty_print=True, encoding='unicode'))

# span digunakaan untuk ads
# p perlu di cek dalam nya jika ada attr target
# div attr target skip
exclude_attrib = ["kompasidRec"]
targe_tag = ["h1","h2","h3","h4","h5","h6","p","img"]
content = []

h1 = xp.xpath("//h1")[0]
text = h1.text
content.append({h1.tag:text})

img= xp.xpath("//div[@class='photo__wrap'] / img")[0]
src= img.xpath("./@src")[0]
alt= img.xpath("./@alt")[0]

content.append({"img":img,"src":src,"alt":alt})
for element in res_div.iter():
    tmp = {}
    tag = element.tag
    attr = element.attrib
    text = element.xpath("string(.)")

    if tag not in targe_tag:
        continue
    if "Baca juga:" in text:
        continue

    tmp[tag] = text.replace("KOMPAS.com", "PT. XYZ")
    content.append(tmp)

print(content)

[{'h1': 'Apa yang Terjadi jika Makan Buah Anggur Setiap Hari?'}, {'img': <Element img at 0x7efd5dbb4460>, 'src': 'https://asset.kompas.com/crops/cZ4iuLqMKP31tyCfl0jIdu9BhnY=/672x448:6048x4032/1200x800/data/photo/2024/10/31/67233b41e8a3a.jpg', 'alt': 'Ilustrasi buah anggur. Apa yang terjadi jika makan buah anggur setiap hari? '}, {'p': 'PT. XYZ - Buah anggur kaya akan antioksidan, vitamin C, dan kalium.'}, {'p': 'Kandungan buah anggur yang terdiri dari varietas berwarna hijau, merah, dan ungu ini juga rendah kalori dan lemak, seperti yang dikutip dari Everyday Health.'}, {'p': 'Jika makan buah anggur setiap hari, Anda bisa mendapatkan beberapa manfaat.'}, {'p': 'Manfaat buah anggur termasuk untuk meningkatkan asupan antioksidan dan kesehatan otak.'}, {'p': 'Artikel ini akan mengulas lebih lengkap tenteng apa yang terjadi jika makan buah anggur setiap hari.'}, {'img': ''}, {'h2': 'Manfaat buah anggur, jika dimakan setiap hari'}, {'p': 'Dikutip dari Eating Well, berikut beberapa potensi m